# **STAT 587 &mdash; Data Science I** &mdash; Homework 2
**Winter 2026**

### Imports & Setup

In [1]:
import sys
from pathlib import Path

# Add project root to Python path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

# Ensure output directories exist
from src.paths import ensure_dirs, DATA_DIR, FIG_DIR, TAB_DIR

ensure_dirs()

In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

from ISLP import load_data

In [3]:
# Set Seed
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [4]:
# GLOBAL OUTPUT CONTROLS

VERBOSE = True           # print / display intermediate outputs
SAVE_FIGS = True         # write figures to reports/figures
SAVE_TABLES = True       # write tables to reports/tables

### Helper Functions

In [5]:
# FLAG HELPERS


# OUTPUT
def vprint(*args, **kwargs):
    if VERBOSE:
        print(*args, **kwargs)


# FIGURES
def save_fig(fig, filename, *, dpi=300):
    """Optionally save matplotlib figure to reports/figures and close it."""
    if SAVE_FIGS:
        path = FIG_DIR / filename
        fig.savefig(path, dpi=dpi, bbox_inches="tight")
        plt.close(fig)
        return path
    plt.close(fig)
    return None


# LaTex TABLES
def latex_escape_underscores(s):
    """Escape underscores for LaTeX."""
    return str(s).replace("_", r"\_")


def escape_df_underscores(df, cols=("Predictor",)):
    """Escape underscores in specified string columns (returns a copy)."""
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = out[c].astype(str).apply(latex_escape_underscores)
    return out


def df_to_tabular_tex(df, *, float_fmt="%.4f", index=True):
    """Return LaTeX tabular with booktabs formatting."""
    return df.to_latex(
        index=index,
        escape=False,
        float_format=(lambda x: float_fmt % x) if float_fmt else None,
        bold_rows=False,
        longtable=False,
    )


def wrap_table(tabular_tex, *, caption, label):
    """Wrap a tabular in a standalone LaTeX table environment."""
    return "\n".join(
        [
            r"\begin{table}[H]",
            r"\begin{center}",
            tabular_tex.strip(),
            r"\end{center}",
            r"\vspace{-5pt}",
            rf"\caption{{{caption}}}",
            rf"\label{{{label}}}",
            r"\end{table}",
            "",
        ]
    )


def save_tex_table(
    df,
    filename,
    *,
    caption,
    label,
    float_fmt="%.4f",
    index=False,
    escape_underscore_cols=None,
):
    """Optionally save a DataFrame as a LaTeX table to reports/tables."""
    if not SAVE_TABLES:
        return None

    if escape_underscore_cols:
        df = escape_df_underscores(df, cols=escape_underscore_cols)

    tex = wrap_table(
        df_to_tabular_tex(df, float_fmt=float_fmt, index=index),
        caption=caption,
        label=label,
    )
    path = TAB_DIR / filename
    path.write_text(tex)
    return path

---
---

## **Question 1:** College Data (ISLP)


Consider the College data from the ISLP package. Details about the data is described on page 65 of the ISLP textbook for this class (https://islp.readthedocs.io/en/latest/datasets/College.html). We would like to *predict* the *number of applications* received using the other variables. 80% of the data (randomly generated) will be treated as training data. The rest will be the test data.

- **Part (a):** Fit a linear model using least squares and report the estimate of the test error.  

- **Part (b):** Fit a tree to the data. Summarize the results. Unless the number of terminal nodes is large, display the tree graphically. Report its MSE.  

- **Part (c):** Use cross-validation to determine whether pruning is helpful and determine the optimal size for the pruned tree. Compare the pruned and un-pruned trees. Report MSE for the pruned tree. Which predictors seem to be the most important?  

- **Part (d):** Use a bagging approach to analyze the data with B = 500 and B = 1000. Compute the MSE. Which predictors seem to be the most important?  

- **Part (e):** Repeat (d) with a random forest approach with B = 500 and B = 1000, and $m \approx p = 3$.  

- **Part (f):** Compare the results from the various methods. Which method would you recommend?

### Q1-Specific Helper Functions and Shared Objects

In [6]:
def importance_side_by_side(
    feature_names,
    importances,
    *,
    title,
    filename_png,
    top_k=12,
    table_caption=None,
    table_label=None,
    table_filename_tex=None,
    show=None,
):
    """
    Create a side-by-side figure:
      - Left: horizontal bar plot of top_k importances
      - Right: a small table (top_k) rendered into the figure
    Optionally also saves a LaTeX table to disk.
    """
    if show is None:
        show = VERBOSE

    
    feature_names = [str(x) for x in feature_names]
    importances = np.asarray(importances)

    assert len(feature_names) == len(importances)

    imp_df = (
        pd.DataFrame({"Predictor": feature_names, "Importance": importances})
        .sort_values("Importance", ascending=False)
        .reset_index(drop=True)
    )

    top_df = imp_df.head(top_k).copy()
    top_df_plot = top_df.iloc[::-1].copy()  # reverse for barh so largest at top visually

    fig, (ax1, ax2) = plt.subplots(
        ncols=2, figsize=(14, 5), gridspec_kw={"width_ratios": [2.2, 1.3]}
    )

    # Bar plot
    ax1.barh(top_df_plot["Predictor"], top_df_plot["Importance"])
    ax1.set_title(title, fontsize=14, fontweight="bold")
    ax1.set_xlabel("Importance", fontsize=12, labelpad=10)
    ax1.tick_params(axis="y", labelsize=9)

    # Table inside plot 
    top_tbl = escape_df_underscores(top_df_plot.copy(), cols=("Predictor",))
    tbl = ax2.table(
        cellText=np.round(top_tbl[["Predictor", "Importance"]].values, 4),
        colLabels=["Predictor", "Importance"],
        loc="center",
        cellLoc="left",
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9)
    tbl.scale(1, 1.2)

    # Optional: save LaTeX table
    if table_filename_tex and table_caption and table_label:
        top_tbl_tex = escape_df_underscores(top_df.copy(), cols=("Predictor",))
        save_tex_table(
            top_tbl_tex,
            table_filename_tex,
            caption=table_caption,
            label=table_label,
            float_fmt="%.4f",
            index=False,
            escape_underscore_cols=None,
        )

    if show:
        plt.show()
    
    save_fig(fig, filename_png)
    
    return top_df

### Data Load, Initial Inspection, Split, Preprocessing

In [7]:
# Load data
q1_df = load_data("College").copy()

In [8]:
# Initial Checks

# Target Column Name
q1_y_col = "Apps"

assert len(q1_df) > 0, "College dataset is empty."
assert q1_y_col in q1_df.columns, f"Target variable '{q1_y_col}' not found."

n_missing = int(q1_df.isna().sum().sum())
assert (
    n_missing == 0
), f"Missing values exist in College data: total missing = {n_missing}"

assert (q1_df[q1_y_col] > 0).all(), "Applications count must be positive."

# Predictors/target
q1_X = q1_df.drop(columns=[q1_y_col])
q1_y = q1_df[q1_y_col].to_numpy()

# Identify categorical vs numeric columns (predictors)
q1_cat_cols = q1_X.select_dtypes(
    include=["object", "category", "bool"]
).columns.tolist()
q1_num_cols = [c for c in q1_X.columns if c not in q1_cat_cols]

assert len(q1_num_cols) + len(q1_cat_cols) == q1_X.shape[1]
assert all(pd.api.types.is_numeric_dtype(q1_X[c]) for c in q1_num_cols)

# Categorical sanity (specific known field)
if "Private" in q1_X.columns:
    assert set(q1_X["Private"].astype(str).unique()) == {
        "Yes",
        "No",
    }, "Unexpected levels in 'Private' variable."

In [9]:
# Initial EDA summary

# Categorical levels
q1_cat_levels = {}
for col in q1_cat_cols:
    q1_cat_levels[col] = sorted(q1_df[col].astype(str).unique().tolist())

q1_cat_levels_str = (
    "; ".join([f"{col}:" + "|".join(levels) for col, levels in q1_cat_levels.items()])
    if q1_cat_levels
    else "(none)"
)

# Numeric ranges (predictors)
q1_num_summary = q1_df[q1_num_cols].describe().T
q1_num_ranges_str = (
    "; ".join(
        [
            f"{col}[{q1_num_summary.loc[col, 'min']:.0f},{q1_num_summary.loc[col, 'max']:.0f}]"
            for col in q1_num_summary.index
        ]
    )
    if len(q1_num_cols) > 0
    else "(none)"
)

# Target summary
q1_apps_summary = q1_df[q1_y_col].describe()
q1_apps_str = (
    f"min={q1_apps_summary['min']:.0f}, "
    f"q1={q1_apps_summary['25%']:.0f}, "
    f"median={q1_apps_summary['50%']:.0f}, "
    f"mean={q1_apps_summary['mean']:.1f}, "
    f"q3={q1_apps_summary['75%']:.0f}, "
    f"max={q1_apps_summary['max']:.0f}"
)

q1_eda_summary = pd.DataFrame(
    {
        "item": [
            "n_rows",
            "n_cols",
            "columns",
            "categorical_cols",
            "categorical_levels",
            "numeric_cols",
            "numeric_ranges_minmax",
            "target",
            "target_summary",
        ],
        "value": [
            q1_df.shape[0],
            q1_df.shape[1],
            ", ".join(q1_df.columns.astype(str).tolist()),
            ", ".join(q1_cat_cols) if q1_cat_cols else "(none)",
            q1_cat_levels_str,
            ", ".join(q1_num_cols) if q1_num_cols else "(none)",
            q1_num_ranges_str,
            q1_y_col,
            q1_apps_str,
        ],
    }
)

if VERBOSE:
    display(q1_eda_summary)

,item,value
0,n_rows,777
1,n_cols,18
2,columns,"Private, Apps, Accept, Enroll, Top10perc, Top2..."
3,categorical_cols,Private
4,categorical_levels,Private:No|Yes
5,numeric_cols,"Accept, Enroll, Top10perc, Top25perc, F.Underg..."
6,numeric_ranges_minmax,"Accept[72,26330]; Enroll[35,6392]; Top10perc[1..."
7,target,Apps
8,target_summary,"min=81, q1=776, median=1558, mean=3001.6, q3=3..."


In [10]:
# Train/test split (80/20)
q1_X_train, q1_X_test, q1_y_train, q1_y_test = train_test_split(
    q1_X, q1_y, test_size=0.2, random_state=SEED
)

# Checks
assert q1_X_train.shape[0] + q1_X_test.shape[0] == q1_df.shape[0]
assert q1_y_train.shape[0] + q1_y_test.shape[0] == q1_df.shape[0]

In [11]:
# Shared Preprocessing
q1_preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", q1_num_cols),
        ("cat", OneHotEncoder(drop="if_binary", handle_unknown="ignore"), q1_cat_cols),
    ],
    remainder="drop",
)

### **1(a)** Least Squares Linear Regression

In [12]:
# Fit a Least Squares Linear Model
q1a_lm = Pipeline(
    steps=[
        ("preprocess", q1_preprocess),
        ("model", LinearRegression()),
    ]
)

q1a_lm.fit(q1_X_train, q1_y_train)
assert hasattr(q1a_lm.named_steps["model"], "coef_"), "Error in lm fit."

# Predictions
q1a_y_train_pred = q1a_lm.predict(q1_X_train)
q1a_y_test_pred = q1a_lm.predict(q1_X_test)

# MSE
q1a_train_mse = mean_squared_error(q1_y_train, q1a_y_train_pred)
q1a_test_mse = mean_squared_error(q1_y_test, q1a_y_test_pred)

assert q1a_train_mse > 0 and q1a_test_mse > 0

In [22]:
# Save Train/Test Error Table (LaTeX)
q1a_mse_tbl = pd.DataFrame(
    {
        "Metric": ["Train MSE", "Test MSE"],
        "Value": [q1a_train_mse, q1a_test_mse],
    }
)

_ = save_tex_table(
    q1a_mse_tbl,
    "q1a_lm_mse.tex",
    caption="Train and test mean squared error (MSE) for the least squares linear regression model.",
    label="tab:q1a-lm-mse",
    float_fmt="%.4f",
    index=False,
)

In [24]:
# Sanity check from large MSEs
q1a_train_rmse = float(np.sqrt(q1a_train_mse))
q1a_test_rmse = float(np.sqrt(q1a_test_mse))

if VERBOSE:    
    print(f"Train RMSE: {q1a_train_rmse}")
    print(f"Test  RMSE:  {q1a_test_rmse}")

Train RMSE: 995.0406856856827
Test  RMSE:  1221.6559986506354


In [23]:
# Extract fitted linear model
q1a_lr = q1a_lm.named_steps["model"]

# Get feature names after preprocessing
q1a_feature_names = q1a_lm.named_steps["preprocess"].get_feature_names_out()

# Build coefficient table
q1a_coef_tbl = pd.DataFrame(
    {
        "Predictor": ["Intercept"] + q1a_feature_names.tolist(),
        "Coefficient": np.concatenate(([q1a_lr.intercept_], q1a_lr.coef_)),
    }
)

# Sanity check
assert q1a_coef_tbl.shape[0] == len(q1a_lr.coef_) + 1

# Optionally save Coefficient Table
_ = save_tex_table(
    q1a_coef_tbl,
    "q1a_lm_coefficients.tex",
    caption="Estimated coefficients for the least squares linear regression model predicting number of applications.",
    label="tab:q1a-lm-coefficients",
    float_fmt="%.4f",
    index=False,
    escape_underscore_cols=("Predictor",),
)

### **2(b)** Regression Tree

In [ ]:
# Fit Unpruned Regression Tree
q1b_tree = Pipeline(
    steps=[
        ("preprocess", q1_preprocess),
        ("model", DecisionTreeRegressor(random_state=SEED)),
    ]
)

q1b_tree.fit(q1_X_train, q1_y_train)
assert hasattr(q1b_tree.named_steps["model"], "tree_")

# Test predictions
q1b_y_test_pred = q1b_tree.predict(q1_X_test)

# Test MSE
q1b_test_mse = mean_squared_error(q1_y_test, q1b_y_test_pred)
assert q1b_test_mse > 0

In [ ]:
# Summarize Tree Size
# Tree summary statistics
q1b_model = q1b_tree.named_steps["model"]

q1b_depth = q1b_model.get_depth()
q1b_n_leaves = q1b_model.get_n_leaves()

# q1b_depth, q1b_n_leaves
# OUTPUT: (21, np.int64(614))
# Too big; don't plot

In [ ]:
# Summarize size in table to justify not plotting
q1b_tree_summary_tbl = pd.DataFrame(
    {
        "Quantity": ["Tree Depth", "Number of Terminal Nodes (Leaves)"],
        "Value": [q1b_depth, int(q1b_n_leaves)],
    }
)

q1b_tree_summary_tex = wrap_table(
    df_to_tabular_tex(q1b_tree_summary_tbl, float_fmt=None, index=False),
    caption="Summary of the unpruned regression tree fit for predicting applications.",
    label="tab:q1b-tree-summary",
)

(TAB_DIR / "q1b_tree_summary.tex").write_text(q1b_tree_summary_tex)

In [ ]:
q1b_mse_tbl = pd.DataFrame(
    {
        "Metric": ["Test MSE"],
        "Value": [q1b_test_mse],
    }
)

q1b_mse_tex = wrap_table(
    df_to_tabular_tex(q1b_mse_tbl, float_fmt="%.4f", index=False),
    caption="Test-set mean squared error (MSE) for the unpruned regression tree.",
    label="tab:q1b-tree-test-mse",
)

(TAB_DIR / "q1b_tree_test_mse.tex").write_text(q1b_mse_tex)

In [ ]:
# Sanity check from large MSE
q1b_test_rmse = np.sqrt(q1b_test_mse)
# q1b_test_rmse

### **(c)** Pruned Regression Tree via Cross-Validation

In [ ]:
#  Cost-Complexity Pruning Path

# Fit Unpruned Tree on Training Data
q1c_tree_unpruned = Pipeline(
    steps=[
        ("preprocess", q1_preprocess),
        ("model", DecisionTreeRegressor(random_state=SEED)),
    ]
)
q1c_tree_unpruned.fit(q1_X_train, q1_y_train)
assert hasattr(q1c_tree_unpruned.named_steps["model"], "tree_")

# Get Transformed Training Matrix to Compute Pruning Path
q1c_X_train_trans = q1c_tree_unpruned.named_steps["preprocess"].transform(q1_X_train)

# Sanity check
assert q1c_X_train_trans.shape[0] == q1_X_train.shape[0]

# Compute Pruning Path for the Fitted (Unpruned) Tree
q1c_unpruned_model = q1c_tree_unpruned.named_steps["model"]
q1c_path = q1c_unpruned_model.cost_complexity_pruning_path(
    q1c_X_train_trans, q1_y_train
)

q1c_ccp_alphas = q1c_path.ccp_alphas
q1c_impurities = q1c_path.impurities

assert q1c_ccp_alphas.ndim == 1
assert len(q1c_ccp_alphas) == len(q1c_impurities)
assert (q1c_ccp_alphas >= 0).all()

In [ ]:
# K-fold CV on Training Data Choose Best alpha
q1c_kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

# Drop Last alpha (stump)
q1c_alphas = q1c_ccp_alphas[:-1]

q1c_cv_rows = []

for a in q1c_alphas:
    q1c_tree = Pipeline(
        steps=[
            ("preprocess", q1_preprocess),
            ("model", DecisionTreeRegressor(random_state=SEED, ccp_alpha=float(a))),
        ]
    )
    # CV MSE (negative in sklearn)
    cv_neg_mse = cross_val_score(
        q1c_tree,
        q1_X_train,
        q1_y_train,
        cv=q1c_kf,
        scoring="neg_mean_squared_error",
    )
    q1c_cv_rows.append(
        {
            "ccp_alpha": float(a),
            "cv_mse_mean": float((-cv_neg_mse).mean()),
            "cv_mse_std": float((-cv_neg_mse).std(ddof=1)),
        }
    )

q1c_cv_results = pd.DataFrame(q1c_cv_rows)

# Choose alpha with Minimum Mean CV MSE
q1c_best_row = q1c_cv_results.loc[q1c_cv_results["cv_mse_mean"].idxmin()]
q1c_best_alpha = float(q1c_best_row["ccp_alpha"])

# Sanity checks
assert q1c_cv_results.shape[0] == len(q1c_alphas)
assert q1c_best_alpha >= 0

In [ ]:
# Check Results
# print(f"q1c_best_alpha:  {q1c_best_alpha}")
# print("\nq1c_cv_results:")
# q1c_cv_results.sort_values("cv_mse_mean").head()

In [ ]:
# CV curve figure
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(
    q1c_cv_results["ccp_alpha"],
    q1c_cv_results["cv_mse_mean"],
    marker="o",
    markersize=3,
    alpha=0.7,
    linestyle="-",
    linewidth=1.5,
)

# Mark Selected alpha
ax.axvline(
    q1c_best_alpha,
    linestyle="--",
    linewidth=2,
    label=r"Selected $\alpha$",
)

ax.set_xscale("log")
ax.set_xlabel(r"Cost-Complexity Parameter $\alpha$", fontsize=12, labelpad=10)
ax.set_ylabel("Mean CV MSE", fontsize=12, labelpad=10)
ax.set_title(
    "Cross-Validation Curve for Cost-Complexity Pruning", fontsize=14, fontweight="bold"
)

ax.legend()
ax.grid(True, which="both", linestyle="--", linewidth=0.5)

# Save figure
fig.savefig(FIG_DIR / "q1c_cv_curve.png", dpi=300, bbox_inches="tight")
plt.close(fig)

In [ ]:
# Fit Pruned Tree using Best alpha

q1c_tree_pruned = Pipeline(
    steps=[
        ("preprocess", q1_preprocess),
        ("model", DecisionTreeRegressor(random_state=SEED, ccp_alpha=q1c_best_alpha)),
    ]
)

q1c_tree_pruned.fit(q1_X_train, q1_y_train)

# Sanity check
assert hasattr(q1c_tree_pruned.named_steps["model"], "tree_")

In [ ]:
# Predict on test set
q1c_y_test_pred = q1c_tree_pruned.predict(q1_X_test)

# Test MSE
q1c_test_mse = mean_squared_error(q1_y_test, q1c_y_test_pred)
assert q1c_test_mse > 0

# RMSE sanity check
q1c_test_rmse = float(np.sqrt(q1c_test_mse))
# q1c_test_rmse

In [ ]:
# Extract Pruned Depth and Num Leaves
q1c_pruned_model = q1c_tree_pruned.named_steps["model"]

q1c_pruned_depth = q1c_pruned_model.get_depth()
q1c_pruned_n_leaves = q1c_pruned_model.get_n_leaves()

assert q1c_pruned_depth >= 0
assert q1c_pruned_n_leaves >= 1

In [ ]:
# Save Pruned Test MSE to LaTex table
q1c_pruned_mse_tbl = pd.DataFrame({"Metric": ["Test MSE"], "Value": [q1c_test_mse]})

q1c_pruned_mse_tex = wrap_table(
    df_to_tabular_tex(q1c_pruned_mse_tbl, float_fmt="%.4f", index=False),
    caption="Test-set mean squared error (MSE) for the pruned regression tree.",
    label="tab:q1c-pruned-tree-test-mse",
)

(TAB_DIR / "q1c_pruned_tree_test_mse.tex").write_text(q1c_pruned_mse_tex)

In [ ]:
# Comparison Table: Unpruned vs Pruned Trees
q1c_tree_comparison_tbl = pd.DataFrame(
    {
        "Quantity": [
            "Tree depth",
            "Number of terminal nodes (leaves)",
            "Test MSE",
        ],
        "Unpruned Tree": [
            q1b_depth,
            int(q1b_n_leaves),
            q1b_test_mse,
        ],
        "Pruned Tree": [
            q1c_pruned_depth,
            int(q1c_pruned_n_leaves),
            q1c_test_mse,
        ],
    }
)

q1c_tree_comparison_tex = wrap_table(
    df_to_tabular_tex(q1c_tree_comparison_tbl, float_fmt="%.4f", index=False),
    caption="Comparison of unpruned and pruned regression trees for predicting applications.",
    label="tab:q1c-tree-comparison",
)

(TAB_DIR / "q1c_tree_comparison.tex").write_text(q1c_tree_comparison_tex)

In [ ]:
# Variable Importance
q1c_feature_names = q1c_tree_pruned.named_steps["preprocess"].get_feature_names_out()
q1c_importances = q1c_tree_pruned.named_steps["model"].feature_importances_

# Sanity checks
assert len(q1c_feature_names) == len(q1c_importances)
assert np.isclose(
    q1c_importances.sum(), 1.0
), f"Feature importances sum to {q1c_importances.sum():.6f}, expected 1.0"
assert (q1c_importances >= 0).all()

q1c_importance_df = pd.DataFrame(
    {"Predictor": q1c_feature_names, "Importance": q1c_importances}
).sort_values("Importance", ascending=False)

In [ ]:
# Variable Importance Table to LaTex
q1c_importance_full = q1c_importance_df.copy()

# Escape underscores for LaTeX
q1c_importance_full["Predictor"] = q1c_importance_full["Predictor"].str.replace(
    "_", r"\_", regex=False
)

q1c_importance_tex = wrap_table(
    df_to_tabular_tex(q1c_importance_full, float_fmt="%.4f", index=False),
    caption="Variable importances from the pruned regression tree. Predictors with zero importance were not used in any split.",
    label="tab:q1c-pruned-tree-importance",
)

(TAB_DIR / "q1c_pruned_tree_importance.tex").write_text(q1c_importance_tex)

In [ ]:
# Pruned Tree Plot: Minimal-labels (feature + threshold only)
q1c_pruned_model = q1c_tree_pruned.named_steps["model"]
q1c_feature_names = q1c_tree_pruned.named_steps["preprocess"].get_feature_names_out()

fig, ax = plt.subplots(figsize=(22, 10))

plot_tree(
    q1c_pruned_model,
    feature_names=q1c_feature_names,
    filled=True,
    rounded=True,
    impurity=False,
    node_ids=False,
    proportion=False,   # no sample proportions
    label="root",       # fewer labels overall
    fontsize=8,
    ax=ax,
)

ax.set_title("Pruned Regression Tree", fontsize=14)
plt.tight_layout()

# Save figure
fig.savefig(FIG_DIR / "q1c_pruned_tree.png", dpi=300, bbox_inches="tight")
plt.close(fig)

### **(d)** Bagging

### **(e)** Random Forest

### **(f)** Method Comparison and Recommendation

---
---

## **Question 2:** Business School Admissions

Consider the business school admission data available in the `admission.csv`. The admission officer of a business school has used an “index” of undergraduate grade point average (GPA, \(X_1\)) and graduate management aptitude test (GMAT, \(X_2\)) scores to help decide which applicants should be admitted to the school’s graduate programs. This index is used to categorize each applicant into one of three groups – admit (group 1), do not admit (group 2), and borderline (group 3).

We will take the last four observations in each category as test data and the remaining observations as training data.

- **Part (a):** Perform an exploratory analysis of the training data by examining appropriate plots and comment on how helpful these predictors may be in predicting response.  

- **Part (b):** Perform an LDA using the training data. Superimpose the decision boundary on an appropriate display of the data. Does the decision boundary seem sensible? In addition, compute the confusion matrix and overall misclassification rate based on both training
and test data. What do you observe?  

- **Part (c):** Repeat (b) using QDA. 

- **Part (d):** Fit a KNN with \(K\) chosen optimally using test error rate. Report error rate, sensitivity, specificity, and AUC for the optimal KNN based on the training data. Also, report its estimated test error rate.  

- **Part (e):** Compare the results in (b), (c) and (d). Which classifier would you recommend? Justify your conclusions.


### Q2-Specific Helper Functions and Shared Objects

In [ ]:
# Helper Function for scatter plots
def plot_q2_scatter(df, ax, order, labels, colors, title=None, alpha=0.7):
    """Scatter plot of GPA vs GMAT colored by admission group."""
    for g in order:
        subset = df[df["Group"] == g]
        ax.scatter(
            subset["GPA"],
            subset["GMAT"],
            label=labels[g],
            color=colors[g],
            alpha=alpha,
            edgecolor="k",
            s=60,
        )
    ax.set_xlabel("Undergraduate GPA", fontsize=12, labelpad=10)
    ax.set_ylabel("GMAT Score", fontsize=12, labelpad=10)
    if title:
        ax.set_title(title, fontsize=14, fontweight="bold")
    ax.legend()
    ax.grid(True)


# Mesh for plotting
def build_gpa_gmat_mesh(df, *, gpa_pad=0.05, gmat_pad=10, n=300):
    gpa_min, gpa_max = df["GPA"].min() - gpa_pad, df["GPA"].max() + gpa_pad
    gmat_min, gmat_max = df["GMAT"].min() - gmat_pad, df["GMAT"].max() + gmat_pad

    xx, yy = np.meshgrid(
        np.linspace(gpa_min, gpa_max, n),
        np.linspace(gmat_min, gmat_max, n),
    )

    grid = np.c_[xx.ravel(), yy.ravel()]
    return xx, yy, grid

In [ ]:
# Color/label mapping for groups
q2_labels = {1: "Admit", 2: "Not Admit", 3: "Border"}
q2_colors = {1: "tab:blue", 2: "tab:orange", 3: "tab:green"}
q2_order = [2, 3, 1]

# Names in order: Not Admit, Border, Admit
q2_group_names = [q2_labels[g] for g in q2_order]

### Data Load, Initial Inspection, Split

In [ ]:
# Load data
q2_df = pd.read_csv(DATA_DIR / "admission.csv")

In [ ]:
# Initial EDA summary
de_levels = sorted(q2_df["De"].astype(str).unique().tolist())
group_levels = sorted(q2_df["Group"].unique().tolist())

class_counts = q2_df["Group"].value_counts().sort_index()

eda_summary = pd.DataFrame(
    {
        "item": [
            "n_rows",
            "n_cols",
            "columns",
            "De_levels",
            "Group_levels",
            "Group_counts",
        ],
        "value": [
            q2_df.shape[0],
            q2_df.shape[1],
            ", ".join(q2_df.columns.astype(str).tolist()),
            ", ".join(de_levels),
            ", ".join(map(str, group_levels)),
            "; ".join([f"{k}:{v}" for k, v in class_counts.items()]),
        ],
    }
)

# eda_summary

In [ ]:
# Sanity check: Verify categorical labels match numeric group coding
category_map = {"admit": 1, "notadmit": 2, "border": 3}

categories_match = (q2_df["De"].map(category_map) == q2_df["Group"]).all()
assert categories_match, "Mismatch between De labels and Group codes."

In [ ]:
# Split:
## Last 4 observations in each category -> test
## Rest of observations -> train

q2_test_idx = q2_df.groupby("Group", sort=False).tail(4).index


q2_train_df = q2_df.drop(index=q2_test_idx).copy()
q2_test_df = q2_df.loc[q2_test_idx].copy()

# Features/labels
q2_X_train = q2_train_df[["GPA", "GMAT"]].to_numpy()
q2_y_train = q2_train_df["Group"].to_numpy()

q2_X_test = q2_test_df[["GPA", "GMAT"]].to_numpy()
q2_y_test = q2_test_df["Group"].to_numpy()

# Checks
assert len(q2_test_df) == 12
assert q2_train_df.shape[0] + q2_test_df.shape[0] == q2_df.shape[0]

### **2(a)** Exploratory Data Analysis

In [ ]:
# Scatter plot: GPA vs GMAT, colored by groups
fig_scatter, ax_scatter = plt.subplots(figsize=(6, 5))

plot_q2_scatter(
    df=q2_train_df,
    ax=ax_scatter,
    order=q2_order,
    labels=q2_labels,
    colors=q2_colors,
    title="Training Data: GPA vs GMAT by Admission Group",
    alpha=0.7,
)

fig_scatter.tight_layout()

# Save figure
fig_scatter.savefig(FIG_DIR / "q2_scatter_gpa_gmat.png", bbox_inches="tight")
plt.close(fig_scatter)

In [ ]:
q2_train_df_plot = q2_train_df.copy()
q2_train_df_plot["GroupOrdered"] = pd.Categorical(
    q2_train_df_plot["Group"].map(q2_labels),
    categories=[q2_labels[g] for g in q2_order],
    ordered=True,
)

fig_box, axes = plt.subplots(1, 2, figsize=(10, 4))

# Boxplot: GPA by Group
q2_train_df_plot.boxplot(column="GPA", by="GroupOrdered", ax=axes[0], grid=False)
axes[0].set_title("GPA by Admission Group", fontsize=13)
axes[0].set_xlabel("Group", fontsize=12, labelpad=10)
axes[0].set_ylabel("GPA", fontsize=12, labelpad=10)

# Boxplot: GMAT by Group
q2_train_df_plot.boxplot(column="GMAT", by="GroupOrdered", ax=axes[1], grid=False)
axes[1].set_title("GMAT by Admission Group", fontsize=13)
axes[1].set_xlabel("Group", fontsize=12, labelpad=10)
axes[1].set_ylabel("GMAT Score", fontsize=12, labelpad=10)

fig_box.suptitle(
    "Training Data: Marginal Distributions by Group", fontsize=16, fontweight="bold"
)
fig_box.tight_layout()


# Save figure
fig_box.savefig(FIG_DIR / "q2_boxplots_gpa_gmat.png", bbox_inches="tight")
plt.close(fig_box)

### **2(b)** Linear Discriminant Analysis (LDA)

In [ ]:
# Fit LDA on training data
q2_lda = LinearDiscriminantAnalysis()
q2_lda.fit(q2_X_train, q2_y_train)

# Predictions
q2_lda_train_pred = q2_lda.predict(q2_X_train)
q2_lda_test_pred = q2_lda.predict(q2_X_test)

# Sanity checks
assert len(q2_lda_train_pred) == len(q2_y_train)
assert len(q2_lda_test_pred) == len(q2_y_test)

In [ ]:
# Build a mesh over predictor space
xx, yy, grid = build_gpa_gmat_mesh(q2_train_df)
Z = q2_lda.predict(grid).reshape(xx.shape)

# Plot decision regions + training points
fig_lda, ax_lda = plt.subplots(figsize=(6.5, 5.5))

# Decision regions
ax_lda.contourf(xx, yy, Z, alpha=0.18)

# Decision boundaries
ax_lda.contour(xx, yy, Z, levels=[1.5, 2.5], colors="k", linewidths=1)

plot_q2_scatter(
    df=q2_train_df,
    ax=ax_lda,
    order=q2_order,
    labels=q2_labels,
    colors=q2_colors,
    title="LDA Decision Regions (Training Data)",
    alpha=0.8,
)

fig_lda.tight_layout()

# Save plot artifact
fig_lda.savefig(FIG_DIR / "q2_lda_boundary.png", bbox_inches="tight")
plt.close(fig_lda)

In [ ]:
# Confusion matrices
# Rows = true class, Columns = predicted class
# Order: Not Admit (2), Border (3), Admit (1)
q2_lda_cm_train = confusion_matrix(q2_y_train, q2_lda_train_pred, labels=q2_order)

q2_lda_cm_test = confusion_matrix(q2_y_test, q2_lda_test_pred, labels=q2_order)

# Overall misclassification rates
q2_lda_train_err = 1.0 - np.mean(q2_lda_train_pred == q2_y_train)
q2_lda_test_err = 1.0 - np.mean(q2_lda_test_pred == q2_y_test)

# Sanity checks
assert q2_lda_cm_train.shape == (3, 3)
assert q2_lda_cm_test.shape == (3, 3)
assert 0.0 <= q2_lda_train_err <= 1.0
assert 0.0 <= q2_lda_test_err <= 1.0

In [ ]:
# Confusion matrices as labeled DataFrames
q2_lda_cm_train_df = pd.DataFrame(
    q2_lda_cm_train, index=q2_group_names, columns=q2_group_names
)
q2_lda_cm_test_df = pd.DataFrame(
    q2_lda_cm_test, index=q2_group_names, columns=q2_group_names
)

# Error-rate summary
q2_lda_metrics_df = pd.DataFrame(
    {
        "Dataset": ["Training", "Test"],
        "Misclassification Rate": [q2_lda_train_err, q2_lda_test_err],
    }
)

# Create LaTex tables
q2_lda_all_tables_tex = "\n".join(
    [
        wrap_table(
            df_to_tabular_tex(q2_lda_cm_train_df, index=True, float_fmt=None),
            caption="LDA confusion matrix (training data). Rows are true classes and columns are predicted classes.",
            label="tab:q2_lda_cm_train",
        ),
        wrap_table(
            df_to_tabular_tex(q2_lda_cm_test_df, index=True, float_fmt=None),
            caption="LDA confusion matrix (test data). Rows are true classes and columns are predicted classes.",
            label="tab:q2_lda_cm_test",
        ),
        wrap_table(
            df_to_tabular_tex(q2_lda_metrics_df, index=False, float_fmt="%.4f"),
            caption="LDA overall misclassification rates.",
            label="tab:q2_lda_error_rates",
        ),
    ]
)

# Write output
(TAB_DIR / "q2_lda_tables.tex").write_text(q2_lda_all_tables_tex, encoding="utf-8")


### **2(c)** Quadratic Discriminant Analysis (QDA)

In [ ]:
# Fit QDA on training data
q2_qda = QuadraticDiscriminantAnalysis()
q2_qda.fit(q2_X_train, q2_y_train)

# Predictions
q2_qda_train_pred = q2_qda.predict(q2_X_train)
q2_qda_test_pred = q2_qda.predict(q2_X_test)

# Sanity checks
assert len(q2_qda_train_pred) == len(q2_y_train)
assert len(q2_qda_test_pred) == len(q2_y_test)

In [ ]:
# Build mesh over predictor space
xx, yy, grid = build_gpa_gmat_mesh(q2_train_df)
Z = q2_qda.predict(grid).reshape(xx.shape)

# Plot decision regions + training points
fig_qda, ax_qda = plt.subplots(figsize=(6.5, 5.5))

# Decision regions
ax_qda.contourf(xx, yy, Z, alpha=0.18)

# Decision boundaries
ax_qda.contour(xx, yy, Z, levels=[1.5, 2.5], colors="k", linewidths=1)

plot_q2_scatter(
    df=q2_train_df,
    ax=ax_qda,
    order=q2_order,
    labels=q2_labels,
    colors=q2_colors,
    title="QDA Decision Regions (Training Data)",
    alpha=0.8,
)

fig_qda.tight_layout()
fig_qda.savefig(FIG_DIR / "q2_qda_boundary.png", bbox_inches="tight")
plt.close(fig_qda)

In [ ]:
# Confusion matrices
# Rows = true class, Columns = predicted class
# Order: Not Admit (2), Border (3), Admit (1)
q2_qda_cm_train = confusion_matrix(q2_y_train, q2_qda_train_pred, labels=q2_order)

q2_qda_cm_test = confusion_matrix(q2_y_test, q2_qda_test_pred, labels=q2_order)

# Misclassification rates
q2_qda_train_err = 1.0 - np.mean(q2_qda_train_pred == q2_y_train)
q2_qda_test_err = 1.0 - np.mean(q2_qda_test_pred == q2_y_test)

# Sanity checks
assert q2_qda_cm_train.shape == (3, 3)
assert q2_qda_cm_test.shape == (3, 3)
assert 0.0 <= q2_qda_train_err <= 1.0
assert 0.0 <= q2_qda_test_err <= 1.0

In [ ]:
# Confusion matrices as labeled DataFrames
q2_qda_cm_train_df = pd.DataFrame(
    q2_qda_cm_train, index=q2_group_names, columns=q2_group_names
)
q2_qda_cm_test_df = pd.DataFrame(
    q2_qda_cm_test, index=q2_group_names, columns=q2_group_names
)

# Error-rate summary
q2_qda_metrics_df = pd.DataFrame(
    {
        "Dataset": ["Training", "Test"],
        "Misclassification Rate": [q2_qda_train_err, q2_qda_test_err],
    }
)

# Create LaTex tables
qda_tables_tex = "\n".join(
    [
        wrap_table(
            df_to_tabular_tex(q2_qda_cm_train_df, float_fmt=None),
            caption="QDA confusion matrix (training data). Rows are true classes and columns are predicted classes.",
            label="tab:q2_qda_cm_train",
        ),
        wrap_table(
            df_to_tabular_tex(q2_qda_cm_test_df, float_fmt=None),
            caption="QDA confusion matrix (test data). Rows are true classes and columns are predicted classes.",
            label="tab:q2_qda_cm_test",
        ),
        wrap_table(
            df_to_tabular_tex(q2_qda_metrics_df, index=False),
            caption="QDA overall misclassification rates.",
            label="tab:q2_qda_error_rates",
        ),
    ]
)

# Write output
(TAB_DIR / "q2_qda_tables.tex").write_text(qda_tables_tex, encoding="utf-8")

### **2(d)** K-Nearest Neighbor (KNN)

In [ ]:
# Choose K by minimizing Test Misclassification Rate

# Odd k to Reduce class tie frequencies
q2_knn_K_values = list(range(1, 26, 2))
q2_knn_test_errors = []

for K in q2_knn_K_values:
    # Use Pipeline to avoid scaling data leakage and scaling errors
    q2_knn = Pipeline(
        [("scalar", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=K))]
    )
    q2_knn.fit(q2_X_train, q2_y_train)
    q2_test_pred = q2_knn.predict(q2_X_test)

    test_err = 1.0 - np.mean(q2_test_pred == q2_y_test)
    q2_knn_test_errors.append(test_err)

q2_knn_results_df = pd.DataFrame(
    {"K": q2_knn_K_values, "test_error_rate": q2_knn_test_errors}
)

# Optimal K (on test error ties, use smallest K)
q2_knn_results_sorted = q2_knn_results_df.sort_values(
    ["test_error_rate", "K"], ascending=[True, True]
)

q2_knn_best_row = q2_knn_results_sorted.iloc[0]

q2_knn_K_star = int(q2_knn_best_row["K"])
q2_knn_test_err_star = float(q2_knn_best_row["test_error_rate"])

# Sanity Checks
assert q2_knn_K_star in q2_knn_K_values
assert 0.0 <= q2_knn_test_err_star <= 1.0

In [ ]:
# Save K-sweep results to LaTex table
q2_knn_results_sorted_rename = q2_knn_results_sorted.rename(
    columns={"test_error_rate": "Test Error Rate"}
)

q2_knn_sweep_tex = wrap_table(
    df_to_tabular_tex(q2_knn_results_sorted_rename, index=False),
    caption="Sorted KNN test error rate (three-class) for candidate values of $K$.",
    label="tab:q2_knn_k_sweep",
)

(TAB_DIR / "q2_knn_k_sweep.tex").write_text(q2_knn_sweep_tex, encoding="utf-8")

In [ ]:
# Fit K* model on full 3-class training data
q2_knn_star = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=q2_knn_K_star)),
    ]
)
q2_knn_star.fit(q2_X_train, q2_y_train)

# 3-class predictions
q2_knn_train_pred3 = q2_knn_star.predict(q2_X_train)
q2_knn_test_pred3 = q2_knn_star.predict(q2_X_test)

# 3-class confusion matrices (ordered as Not Admit, Border, Admit)
q2_knn_cm_train3 = confusion_matrix(q2_y_train, q2_knn_train_pred3, labels=q2_order)
q2_knn_cm_test3 = confusion_matrix(q2_y_test, q2_knn_test_pred3, labels=q2_order)

# 3-class misclassification rates
q2_knn_train_err3 = 1.0 - np.mean(q2_knn_train_pred3 == q2_y_train)
q2_knn_test_err3 = 1.0 - np.mean(q2_knn_test_pred3 == q2_y_test)

# Sanity Checks
assert q2_knn_cm_train3.shape == (3, 3)
assert q2_knn_cm_test3.shape == (3, 3)
assert 0.0 <= q2_knn_train_err3 <= 1.0
assert 0.0 <= q2_knn_test_err3 <= 1.0

In [ ]:
# Confusion matrices as labeled DataFrames
q2_knn_cm_train3_df = pd.DataFrame(
    q2_knn_cm_train3, index=q2_group_names, columns=q2_group_names
)
q2_knn_cm_test3_df = pd.DataFrame(
    q2_knn_cm_test3, index=q2_group_names, columns=q2_group_names
)

# Error-rate summary
q2_knn_err3_df = pd.DataFrame(
    {
        "Dataset": ["Training", "Test"],
        "Misclassification Rate": [q2_knn_train_err3, q2_knn_test_err3],
    }
)

# Create LaTex tables
q2_knn_tables3_tex = "\n".join(
    [
        wrap_table(
            df_to_tabular_tex(q2_knn_cm_train3_df, float_fmt=None),
            caption="KNN confusion matrix at $K^*$ (training data). Rows are true classes and columns are predicted classes.",
            label="tab:q2_knn_cm_train3",
        ),
        wrap_table(
            df_to_tabular_tex(q2_knn_cm_test3_df, float_fmt=None),
            caption="KNN confusion matrix at $K^*$ (test data). Rows are true classes and columns are predicted classes.",
            label="tab:q2_knn_cm_test3",
        ),
        wrap_table(
            df_to_tabular_tex(q2_knn_err3_df, index=False),
            caption="KNN misclassification rates (three-class) at $K^*$.",
            label="tab:q2_knn_err3",
        ),
    ]
)

# Write output
(TAB_DIR / "q2_knn_3class_tables.tex").write_text(q2_knn_tables3_tex, encoding="utf-8")

In [ ]:
# Multiclass OvR metrics on training data for KNN at K*

# Confusion Matrix Totals
n = q2_knn_cm_train3.sum()
rowsum = q2_knn_cm_train3.sum(axis=1)  # actual totals per class
colsum = q2_knn_cm_train3.sum(axis=0)  # predicted totals per class
diag = np.diag(q2_knn_cm_train3)

# OvR per-class sensitivity and specificity
sens = diag / rowsum
spec = (n - rowsum - colsum + diag) / (n - rowsum)

# Macro averages
sens_macro = sens.mean()
spec_macro = spec.mean()

# Multiclass AUC (OvR macro) using predicted probabilities
proba_train = q2_knn_star.predict_proba(q2_X_train)

# Align columns to labels [1,2,3] for `roc_auc_score()`
proba_df = pd.DataFrame(proba_train, columns=q2_knn_star.named_steps["knn"].classes_)
proba_aligned = proba_df[[1, 2, 3]].to_numpy()

q2_knn_auc_ovr_macro = roc_auc_score(
    q2_y_train, proba_aligned, multi_class="ovr", average="macro"
)

# Package for table
q2_knn_multiclass_df = pd.DataFrame(
    {
        "Class": q2_group_names,
        "Sensitivity (OvR)": sens,
        "Specificity (OvR)": spec,
    }
)

q2_knn_multiclass_summary_df = pd.DataFrame(
    {
        "Metric": ["Macro sensitivity", "Macro specificity", "AUC (OvR macro)"],
        "Value": [sens_macro, spec_macro, q2_knn_auc_ovr_macro],
    }
)

# Sanity checks
assert q2_knn_multiclass_df.shape == (3, 3)
assert 0.0 <= sens_macro <= 1.0
assert 0.0 <= spec_macro <= 1.0
assert 0.0 <= q2_knn_auc_ovr_macro <= 1.0

In [ ]:
q2_knn_multiclass_tex = "\n".join(
    [
        wrap_table(
            df_to_tabular_tex(q2_knn_multiclass_df, float_fmt="%.4f", index=False),
            caption="KNN multiclass one-vs-rest (OvR) sensitivity and specificity by class (training data) at $K^*$.",
            label="tab:q2_knn_ovr_by_class",
        ),
        wrap_table(
            df_to_tabular_tex(
                q2_knn_multiclass_summary_df, float_fmt="%.4f", index=False
            ),
            caption="KNN multiclass OvR macro-averaged metrics (training data) at $K^*$.",
            label="tab:q2_knn_ovr_summary",
        ),
    ]
)

(TAB_DIR / "q2_knn_multiclass_metrics.tex").write_text(
    q2_knn_multiclass_tex, encoding="utf-8"
)

In [ ]:
# Final KNN summary table

q2_knn_summary_df = pd.DataFrame(
    {
        "Quantity": [
            r"$K^*$",
            "Training error rate",
            "Training sensitivity (OvR, macro)",
            "Training specificity (OvR, macro)",
            "Training AUC (OvR, macro)",
            "Test error rate",
        ],
        "Value": [
            q2_knn_K_star,
            q2_knn_train_err3,
            sens_macro,
            spec_macro,
            q2_knn_auc_ovr_macro,
            q2_knn_test_err3,
        ],
    }
)

q2_knn_summary_tex = wrap_table(
    df_to_tabular_tex(q2_knn_summary_df, float_fmt="%.4f", index=False),
    caption="Summary of optimal KNN performance.",
    label="tab:q2_knn_summary",
)

(TAB_DIR / "q2_knn_summary.tex").write_text(q2_knn_summary_tex, encoding="utf-8")

### **2(e)** Method Comparison and Recommendation

In [ ]:
q2_model_comparison_df = pd.DataFrame(
    {
        "Model": ["LDA", "QDA", "KNN (K=3)"],
        "Training Error Rate": [
            q2_lda_train_err,
            q2_qda_train_err,
            q2_knn_train_err3,
        ],
        "Test Error Rate": [
            q2_lda_test_err,
            q2_qda_test_err,
            q2_knn_test_err3,
        ],
    }
)

q2_model_comparison_tex = wrap_table(
    df_to_tabular_tex(q2_model_comparison_df, index=False),
    caption="Comparison of training and test misclassification rates for LDA, QDA, and KNN.",
    label="tab:q2_model_comparison",
)

(TAB_DIR / "q2_model_comparison.tex").write_text(
    q2_model_comparison_tex, encoding="utf-8"
)